In [1]:
!pip install transformers

In [2]:
conda install pytorch torchvision cudatoolkit=11.2 -c pytorch

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Note: you may need to restart the kernel to use updated packages.




PackagesNotFoundError: The following packages are not available from current channels:

  - cudatoolkit=11.2

Current channels:

  - https://conda.anaconda.org/pytorch/win-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [26]:
!pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch: started
    Running setup.py install for pytorch: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\howto\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\howto\\AppData\\Local\\Temp\\pip-install-iofw0n9r\\pytorch_b5bc674ad8764c069e9d391db0c93729\\setup.py'"'"'; __file__='"'"'C:\\Users\\howto\\AppData\\Local\\Temp\\pip-install-iofw0n9r\\pytorch_b5bc674ad8764c069e9d391db0c93729\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\howto\AppData\Local\Temp\pip-wheel-b03c9v1q'
       cwd: C:\Users\howto\AppData\Local\Temp\pip-install-iofw0n9r\pytorch_b5bc674ad8764c069e9d391db0c93729\
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\howto\AppData\Local\Temp\pip-install-iofw0n9r\pytorch_b5bc674ad8764c069e9d391db0c93729\setup.py", line 15, in <module>
     

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from transformers import AutoModel
from transformers import AutoTokenizer

In [18]:
train_path = 'gedichte.txt'

In [19]:
from transformers import TextDataset, DataCollatorForLanguageModeling
# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

C:\Users\howto\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2Tokenizer_64_gedichte.txt [took 0.001 s]


In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [12]:
trainer.train()

***** Running training *****
  Num examples = 19
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.12120740890502929, metrics={'train_runtime': 1392.8561, 'train_samples_per_second': 1.364, 'train_steps_per_second': 0.072, 'total_flos': 62056857600000.0, 'train_loss': 0.12120740890502929, 'epoch': 100.0})

In [21]:
text = "Сияет небо"
input_ids = tokenizer.encode(text, return_tensors="pt")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Сияет небо и озаряется Луна.
И звёзды светясь отражают всё прекрасное,
В Мир свой приход озаряясь полной святостью.

С добротой делитый, я жил
Во знойных степях,
В селениях юных,
И было со мной так душевно,
Что мог бы пожелать
О смерти даже небожеский род!

Рассказал я вам сон,
Двум деткам на заметку пришитый
И очень ему зацвенявшей,
Палачей по пояс.
Первый, как показалось глазам, замер,
Вздохнув без отдыха. Едва живой,
Проливший кровь, валялся старик во гробу,
Двух детей уложив наземь.
В тот же миг рядышком сомкнулся
Рог с воинственным кличем: "Вiдэс!" - и мёртвым он впопыхах
Двоих обезображил сталью.


In [22]:
text = "Конец пути"
input_ids = tokenizer.encode(text, return_tensors="pt")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Конец пути». Это отрывок из песни О. Уайльда "Брат мой Алексей, не плачь". На поле битвы человек встает и идёт к дому. Навстречу ему — Лиза, вся заляпанная мутью. На коленях стоит, причитая она. Спит она в избе рядом с ним. Лежал он один, как мёртвый, в избе. Спала она в избе, у Ивана Васильевича... Всё кругом затихло... Встал богатырь — и видит: перед ними стоит дом родной. Заплакал и обмер от жалости человек. Бросил он к ногам царь царей дары: рубаху да штаны отдал, велел слугам рубить с убитых, да жарить в печи...

Через лес прошёл человек большой — большой! — и стал копать яму. Но не шелохнулись распоротые одежды. Из земли показались на свет сухие черви. Стал злодеи совокупляться — человек рыдал всё громче и свирепей. Наконец,


In [17]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
Между гор седыми
И тающая сага
За облачко чердачное
В окошко видно.
И куда б ни зашел вечер,
Как огонек горит, как свечка горит,
Там, в доме, где жить нам всем теперь надо,
В домике, полном тепла и сил,
Он с нами опять на посиделки
У камина сидят.
Дивятся, умиляются детвора
Разгулу веселых рук,
И шумно смеются, звонко
Ребенку гордый стан;
И жарко им, и весело,
На солено боронке
Взлаять друг друга, поиграть.
Да дети кругом шумят,
И беседа светская в комнате 
От избытка кружит;
И шумят веселием игра,
Лихо катя, колесом скачет,
Меж горстых изб тесня, -
Чтоб шумел, как белка, водич


1)общее: https://link.springer.com/article/10.1007/BF00279189 <br>
    GPT-1:http://ceur-ws.org/Vol-2380/paper_102.pdf <br>
    GPT-2:https://dl.acm.org/doi/pdf/10.1145/3460426.3463662 <br>
    GPT-3: https://www.sciencedirect.com/science/article/pii/S2667325821002193<br>
    
2)
Размеры модели значительно увеличились за счет набора данных Common Crawl — это веб-архив, собранный с 2011 года и состоящий из почти триллиона слов. Чтобы улучшить среднее качество наборов данных, разработчики сделали следующее:
> - отфильтровали версию CommonCrawl (веб-архив, собранный с 2011 года и состоящий из почти триллиона слов);
> - выполнили нечеткую дедубликацию на уровне документа (внутри) и между наборами данных, чтобы предотвратить избыточность и сохранить целостность валидационного набора данных, применяемого для оценки переобучения;
> - для разнообразия добавили известные высококачественные эталонные корпуса в набор тренировочных данных: WebText; корпуса книг Books1 и Books2; англоязычную Википедию.